In [1]:
from hypore import *

In [2]:
# create state vector with 4 components and derivatives up to order 2
# and make available in global namespace
xx = gen_state(2,3)
xx.T

[x₁  x₂  ẋ₁  ẋ₂  ẍ₁  ẍ₂  ẍ̇₁  ẍ̇₂]

In [3]:
A0 = sp.Matrix([
    [1,xdot1],
    [x2,x1*x2]])
A1 = sp.Matrix([
    [1,x1],
    [x2,0]])
A2 = sp.Matrix([
    [1,0],
    [0,0]])
A0,A1,A2 # A(d/dt) = A0 + A1*d/dt + A2*(d/dt)**2

⎛⎡1    ẋ₁ ⎤, ⎡1   x₁⎤, ⎡1  0⎤⎞
⎜⎢         ⎥  ⎢      ⎥  ⎢    ⎥⎟
⎝⎣x₂  x₁⋅x₂⎦  ⎣x₂  0 ⎦  ⎣0  0⎦⎠

In [4]:
# manually:
# st.rnd_number_rank(spread(A0,A1,beta=0)) \
# == st.rnd_number_rank(spread_aug(A0,A1,beta=0)) \
# and st.rnd_number_rank(spread(A0,A1,beta=0)) == spread(A0,A1,beta=0).shape[0]

In [5]:
# st.rnd_number_rank(spread(A0,A1,beta=1)) \
# == st.rnd_number_rank(spread_aug(A0,A1,beta=1)) \
# and st.rnd_number_rank(spread(A0,A1,beta=1)) == spread(A0,A1,beta=1).shape[0]

In [6]:
# st.rnd_number_rank(spread(A0,A1,beta=2)) \
# == st.rnd_number_rank(spread_aug(A0,A1,beta=2)) \
# and st.rnd_number_rank(spread(A0,A1,beta=2)) == spread(A0,A1,beta=2).shape[0]

In [7]:
# automated:
# returns degree beta of inverse (-1 => not unimodular)
beta_ = is_unimodular(A0,A1,A2)
if (beta_==-1):
    print("A is not unimodular")
else:
    print("A is unimodular")

A is unimodular


In [8]:
beta_

2

In [9]:
sA2 = Top(A0,A1,A2,beta=beta_)
sA2

⎡  1.0                   1.0⋅ẋ₁                        1.0                 1.
⎢                                                                             
⎢1.0⋅x₂                 1.0⋅x₁⋅x₂                     1.0⋅x₂                  
⎢                                                                             
⎢   0                    1.0⋅ẍ₁                        1.0                 2.
⎢                                                                             
⎢1.0⋅ẋ₂         1.0⋅x₁⋅ẋ₂ + 1.0⋅x₂⋅ẋ₁         1.0⋅x₂ + 1.0⋅ẋ₂          1.0
⎢                                                                             
⎢   0                   1.0⋅ẍ̇₁                         0                  3.
⎢                                                                             
⎣1.0⋅ẍ₂  1.0⋅x₁⋅ẍ₂ + 1.0⋅x₂⋅ẍ₁ + 2.0⋅ẋ₁⋅ẋ₂  1.0⋅ẍ₂ + 2.0⋅ẋ₂  2.0⋅x₁⋅ẋ₂

0⋅x₁                 1.0             0        0       0      0   0⎤
                                                              

In [10]:
sA2_rpinv = right_pseudo_inverse(sA2)
sA2_rpinv.shape

(10, 6)

In [11]:
I, O = sp.eye(2), sp.zeros(2,8)
IO = st.col_stack(I,O)
B_ = IO*sA2_rpinv
B_ = sp.simplify(B_)

In [12]:
B_

⎡                            1.0⋅ẋ₂                                       -1.
⎢ 1.0                        ───────                          0            ───
⎢                                2                                           x
⎢                              x₂                                             
⎢                                                                             
⎢           ⎛      2                                    2⎞                    
⎢-1.0   1.0⋅⎝1.0⋅x₂  - 1.0⋅x₂⋅ẍ₂ - 1.0⋅x₂⋅ẋ₂ + 2.0⋅ẋ₂ ⎠  -1.0   1.0⋅(1.0⋅x₂
⎢─────  ──────────────────────────────────────────────────  ─────  ───────────
⎢  x₁                              3                          x₁              
⎣                             x₁⋅x₂                                        x₁⋅

0                    ⎤
──           0    0  ⎥
₂                    ⎥
                     ⎥
                     ⎥
                     ⎥
 - 2.0⋅ẋ₂)      1.0 ⎥
───────────  0  ─────⎥
  2             x₁⋅x₂⎥
x₂

In [13]:
B_.shape

(2, 6)

In [14]:
B0 = st.col_select(B_, 0,1)
B1 = st.col_select(B_, 2,3)
B2 = st.col_select(B_, 4,5)
# unimodular inverse of A(d/dt):
B0, B1, B2 # B(d/dt) = B0 + B1*d/dt + B2*(d/dt)**2

⎛⎡                            1.0⋅ẋ₂                      ⎤  ⎡               
⎜⎢ 1.0                        ───────                      ⎥, ⎢  0            
⎜⎢                                2                        ⎥  ⎢               
⎜⎢                              x₂                         ⎥  ⎢               
⎜⎢                                                         ⎥  ⎢-1.0   1.0⋅(1.0
⎜⎢           ⎛      2                                    2⎞⎥  ⎢─────  ────────
⎜⎢-1.0   1.0⋅⎝1.0⋅x₂  - 1.0⋅x₂⋅ẍ₂ - 1.0⋅x₂⋅ẋ₂ + 2.0⋅ẋ₂ ⎠⎥  ⎢  x₁           
⎜⎢─────  ──────────────────────────────────────────────────⎥  ⎣               
⎜⎢  x₁                              3                      ⎥                  
⎝⎣                             x₁⋅x₂                       ⎦                  

-1.0          ⎤            ⎞
─────         ⎥, ⎡0    0  ⎤⎟
  x₂          ⎥  ⎢        ⎥⎟
              ⎥  ⎢    1.0 ⎥⎟
⋅x₂ - 2.0⋅ẋ₂)⎥  ⎢0  ─────⎥⎟
──────────────⎥  ⎣   x₁⋅x₂⎦⎟
     2        ⎥            ⎟
x₁⋅x₂ 